In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import regex as re

In [2]:
# Load the data
df = pd.read_csv('./Best_Sellers_top100_2dep_other_info.csv')
# Check the data
df

,Unnamed: 0,all_rank_list,format,before_discount,new_price,title,scrape_time,other_list
0,0,"#30,397","Audio CD – Audiobook, November 1, 2004",_,$15.23,The Adventure Begins: The Early Classics (Adve...,2024-04-11 23:08:05.609874,Publisher:Tyndale Entertainment; Abridged ed...
1,1,"#54,341",1st Edition,$22.99,$12.28 - $17.84,Spanish For Dummies Audio Set,2024-04-11 23:08:15.497661,Publisher:For Dummies; 1st edition (April 30...
2,2,"#29,496","Audio CD – CD, October 4, 2022",$14.99,$10.99,"28 Hours: 6 stories on Trust, Truth, and Disce...",2024-04-11 23:08:24.228518,"Publisher:Focus on the Family (October 4, 20..."
3,3,"#43,543","Audio CD – January 1, 2016",$13.98,$10.49,24K Magic,2024-04-11 23:08:32.071115,ASIN:B01LYRM74T Publisher:Atlantic (Januar...
4,4,#5,Kindle Edition,_,$14.99 (function(f) {var _np=(window.P._names...,The Women: A Novel,2024-04-11 23:08:46.121043,ASIN:B0C1X97LW7 Publisher:St. Martin's Pre...
...,...,...,...,...,...,...,...,...
73478,73478,"#25,849","Paperback – August 18, 2021",_,$14.99,Florida Bucket List Adventure Guide & Journal:...,2024-04-12 02:10:10.926703,"Publisher:Bridge Press (August 18, 2021) L..."
73479,73479,"#103,058","Paperback – February 4, 2020",_,$9.99,The New York Times Blooming Book of Mini Cross...,2024-04-12 02:10:59.091063,"Publisher:St. Martin's Griffin (February 4, ..."
73480,73480,"#15,563","Paperback – Illustrated, August 1, 2016",$27.95,$22.95,Plants of the Pacific Northwest Coast: Washing...,2024-04-12 02:11:09.792930,Publisher:Lone Pine International; Revised e...
73481,73481,"#1,451,928",Mass Market Paperback,_,$35.84,Plants of the Pacific Northwest Coast: Washing...,2024-04-12 02:11:16.144042,ASIN:B01FGOFI56


In [3]:
#data cleaning
temp = df['before_discount'][1][0]

before_discount = []
for index,row in df.iterrows():
    data = row['before_discount'].replace(temp,'').replace('_','').replace('Digital List Price:   ','').replace('\n','').replace('Print List Price:  ','').replace(',','')
    if data == '':
        data = np.nan
    before_discount.append(data)
df['before_discount'] = before_discount
df['before_discount'] = pd.to_numeric(df['before_discount'], errors='coerce')



new_price = []
for index,row in df.iterrows():
    data = re.sub('[^0-9.]', '', row['new_price'].replace(temp,'').replace('_','').replace(',','')[0:5])
    if data == '':
        data = np.nan
    new_price.append(data)
df['new_price'] = new_price
df['before_discount'] = pd.to_numeric(df['before_discount'], errors='coerce')

format = []
publish_date = []
for index,row in df.iterrows():
    data = row['format'].split(' – ')
    format.append(data[0])
    match = re.search('\((.*?)\)', row['other_list'])
    if match:
        publish_date.append(match.group(1))
    else:
        publish_date.append(np.nan)
df['format'] = format
df['publish_date'] = publish_date

df['publish_date'] = pd.to_datetime(df['publish_date'], format="%B %d, %Y", errors='coerce').dt.strftime("%Y-%m-%d")
# Replace 'NaT' with np.nan
df['publish_date'].replace('NaT', np.nan, inplace=True)




/var/folders/3p/81nnp2994m19w63h0mm8m4bc0000gn/T/ipykernel_33029/3821165576.py:39: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['publish_date'].replace('NaT', np.nan, inplace=True)


In [4]:
mask1= df['before_discount'].isnull()
mask2 = df['new_price'].isnull()
mask = mask1 | mask2
clean_df = df[~mask]


In [5]:
titledf = clean_df
titledf = titledf.drop_duplicates(subset=['title'])
titledf = titledf.reset_index(drop=True)
titledf = titledf[['title','publish_date']]
paperdf = clean_df[clean_df['format'] == 'Paperback']
harddf = clean_df[clean_df['format'] == 'Hardcover']
ebookdf = clean_df[clean_df['format'] == 'Kindle Edition']

paper_original_price = []
paper_new_price = []
hand_original_price = []
hand_new_price = []
paperlist = paperdf['title'].tolist()
handlist = harddf['title'].tolist()

for index1,row1 in titledf.iterrows():
    print(f'{index1}/{len(titledf)}')
    if row1['title'] in paperlist:
        print('exist')
        for index2,row2 in paperdf.iterrows():
            if row1['title'] == row2['title']:
                paper_original_price.append(row2['before_discount'])
                paper_new_price.append(row2['new_price'])
                break
    else:
        paper_original_price.append(np.nan)
        paper_new_price.append(np.nan)
        
    if row1['title'] in handlist:
        print('exist')
        for index3,row3 in harddf.iterrows():
            if row1['title'] == row3['title']:
                hand_original_price.append(row3['before_discount'])
                hand_new_price.append(row3['new_price'])
                break
    else:
        hand_original_price.append(np.nan)
        hand_new_price.append(np.nan)
    
    
titledf['paper_original_price'] = paper_original_price
titledf['paper_new_price'] = paper_new_price
titledf['hand_original_price'] = hand_original_price
titledf['hand_new_price'] = hand_new_price



0/22611
1/22611
2/22611
3/22611
exist
4/22611
5/22611
6/22611
7/22611
8/22611
9/22611
10/22611
11/22611
12/22611
13/22611
14/22611
15/22611
16/22611
17/22611
18/22611
exist
19/22611
exist
exist
20/22611
exist
exist
21/22611
22/22611
23/22611
exist
exist
24/22611
25/22611
exist
26/22611
exist
exist
27/22611
exist
28/22611
exist
29/22611
30/22611
31/22611
32/22611
33/22611
34/22611
exist
35/22611
36/22611
37/22611
38/22611
exist
39/22611
40/22611
41/22611
42/22611
43/22611
44/22611
45/22611
46/22611
47/22611
exist
exist
48/22611
exist
exist
49/22611
exist
50/22611
exist
51/22611
exist
52/22611
53/22611
54/22611
55/22611
56/22611
exist
exist
57/22611
exist
58/22611
exist
59/22611
exist
60/22611
exist
exist
61/22611
62/22611
63/22611
64/22611
exist
65/22611
exist
exist
66/22611
67/22611
exist
68/22611
69/22611
exist
70/22611
exist
71/22611
exist
72/22611
exist
73/22611
exist
74/22611
exist
75/22611
exist
exist
76/22611
77/22611
exist
78/22611
exist
79/22611
exist
exist
80/22611
exist
81/22

KeyboardInterrupt: 

In [ ]:
# convert to float
vaild_df = titledf.dropna(subset=['paper_original_price','paper_new_price','hand_original_price','hand_new_price'])
vaild_df['paper_new_price'] = vaild_df['paper_new_price'].astype(float)
vaild_df['paper_original_price'] = vaild_df['paper_original_price'].astype(float)
vaild_df['hand_new_price'] = vaild_df['hand_new_price'].astype(float)
vaild_df['hand_original_price'] = vaild_df['hand_original_price'].astype(float)

# calculate discount rate
vaild_df['paper_discount_rate'] = (vaild_df['paper_new_price']/vaild_df['paper_original_price'])*100
vaild_df['hand_discount_rate'] = (vaild_df['hand_new_price']/vaild_df['hand_original_price'])*100
vaild_df['relative_discount_hand_to_paper'] = 1/(vaild_df['hand_discount_rate']/vaild_df['paper_discount_rate'])*100
vaild_df['relative_price_hand_to_paper'] = 1/(vaild_df['hand_new_price']/vaild_df['paper_new_price'])*100
vaild_df['relative_original_price_hand_to_paper'] = 1/(vaild_df['hand_original_price']/vaild_df['paper_original_price'])*100

In [ ]:
# make  kedplot of paper discount rate and hand discount rate 
plt.figure(figsize=(10,6))
sns.kdeplot(vaild_df['paper_discount_rate'], label='Paperback')
sns.kdeplot(vaild_df['hand_discount_rate'], label='Hardcover')
plt.title('Discount Rate Distribution')
plt.xlabel('Discount Rate (%)')
plt.ylabel('Density')
plt.legend()
plt.show()


In [ ]:
vaild_df['publish_date'] = vaild_df['publish_date'].astype('datetime64[ns]')
vaild_df['publish_date_from_min'] = pd.to_numeric((vaild_df['publish_date'] - vaild_df['publish_date'].max()).dt.days, errors='coerce')
vaild_df = vaild_df.dropna(subset=['publish_date_from_min'])
fig,ax = plt.subplots(3,2,figsize=(50,30))
#---------------------

ax[0,0].scatter(vaild_df['publish_date_from_min'],vaild_df['paper_discount_rate'])
z = np.polyfit(vaild_df['publish_date_from_min'], vaild_df['paper_discount_rate'], 1)
p = np.poly1d(z)
ax[0,0].plot(vaild_df['publish_date_from_min'],p(vaild_df['publish_date_from_min']),"r--")

ax[0,0].set_title('Paper Discount Rate by Publish Date',fontsize = 30)
ax[0,0].set_xlabel('Days from the most recent publish date')
ax[0,0].set_ylabel('Discount Rate (%)')
#---------------------
ax[0,1].scatter(vaild_df['publish_date_from_min'],vaild_df['hand_discount_rate'])
z = np.polyfit(vaild_df['publish_date_from_min'], vaild_df['hand_discount_rate'], 1)
p = np.poly1d(z)
ax[0,1].plot(vaild_df['publish_date_from_min'],p(vaild_df['publish_date_from_min']),"r--")

ax[0,1].set_title('Hardcover Discount Rate by Publish Date',fontsize = 30)
ax[0,1].set_xlabel('Days from the most recent publish date')
ax[0,1].set_ylabel('Discount Rate (%)')
#---------------------
ax[1,0].scatter(vaild_df['publish_date_from_min'],vaild_df['relative_discount_hand_to_paper'])
z = np.polyfit(vaild_df['publish_date_from_min'], vaild_df['relative_discount_hand_to_paper'], 1)
p = np.poly1d(z)
ax[1,0].plot(vaild_df['publish_date_from_min'],p(vaild_df['publish_date_from_min']),"r--")

ax[1,0].set_title('paperback discount / hardcover discount by Publish Date',fontsize = 30)
ax[1,0].set_xlabel('Days from the most recent publish date')
ax[1,0].set_ylabel('paperback discount / hardcover discount Rate (%)')
#---------------------
ax[1,1].scatter(vaild_df['publish_date_from_min'],vaild_df['relative_price_hand_to_paper'])
z = np.polyfit(vaild_df['publish_date_from_min'], vaild_df['relative_price_hand_to_paper'], 1)
p = np.poly1d(z)
ax[1,1].plot(vaild_df['publish_date_from_min'],p(vaild_df['publish_date_from_min']),"r--")

ax[1,1].set_title(' Hardcover price / Paperback price by Publish Date',fontsize = 30)
ax[1,1].set_xlabel('Days from the most recent publish date')
ax[1,1].set_ylabel('paperback price / hardcover price Rate (%)')
#---------------------
ax[2,0].scatter(vaild_df['publish_date_from_min'],vaild_df['relative_original_price_hand_to_paper'])
z = np.polyfit(vaild_df['publish_date_from_min'], vaild_df['relative_original_price_hand_to_paper'], 1)
p = np.poly1d(z)
ax[2,0].plot(vaild_df['publish_date_from_min'],p(vaild_df['relative_original_price_hand_to_paper']),"r--")

ax[2,0].set_title(' paperback original price / hardcover original price  by Publish Date',fontsize = 30)
ax[2,0].set_xlabel('Days from the most recent publish date')
ax[2,0].set_ylabel('paperback original price / hardcover original price  Rate (%)')

plt.savefig('./pics/Discount Rate by Publish Date.png')

# explian the picture in chinese
# 1. Paperback Discount Rate by Publish Date
# 2. Hardcover Discount Rate by Publish Date
# 3. paperback discount / hardcover discount by Publish Date
# 4. Relative Price between Hardcover and Paperback by Publish Date
# 5. paperback original price / hardcover original price  by Publish Date




In [ ]:
_df = pd.concat([paperdf,harddf])
# convert to days from now
_df['publish_date'] = pd.to_datetime(_df['publish_date'], format="%Y-%m-%d", errors='coerce')
_df['publish_date_from_min'] = pd.to_numeric((_df['publish_date'] - _df['publish_date'].max()).dt.days, errors='coerce')
_df = _df.dropna(subset=['publish_date_from_min'])
_df = _df[_df['publish_date_from_min'] > -20000]
# use kdeensity plot to show the distribution of book by publish date
plt.figure(figsize=(30,18))
sns.kdeplot(data = _df[_df['format'] == 'Paperback'], x='publish_date_from_min',  fill=True,label = 'Paperback')
sns.kdeplot(data = _df[_df['format'] == 'Hardcover'], x='publish_date_from_min',  fill=True, label = 'Hardcover')

plt.title('Book Distribution by Publish Date',fontsize = 30)
plt.xlabel('Days from the most recent publish date')
plt.ylabel('Density')
plt.legend(loc='upper left')
plt.savefig('./pics/Book Distribution by Publish Date.png')

# explian picture in chinese
# Book Distribution by Publish Date
# 1. Paperback
# 2. Hardcover




In [ ]:

plt.figure(figsize=(30,30))
vaild_df['nyears'] = vaild_df['publish_date'].dt.year
vaild_df= vaild_df[vaild_df['nyears'] > 2000]
vaild_df = vaild_df[vaild_df['hand_new_price']<60]
vaild_df = vaild_df[vaild_df['paper_new_price']<60]
vaild_df['nyears_group'] = (vaild_df['nyears'] // 5) * 5




sns.kdeplot(data = vaild_df,x = 'paper_new_price',y = 'hand_new_price',hue = 'nyears_group',thresh=0.1,levels=10)
# draw 45 degree line
plt.plot([0,60],[0,60],'r--')

plt.title('Price Comparison between Hardcover and Paperback',fontsize = 30)
plt.xlabel('Paperback Price',fontsize = 20)
plt.ylabel('Hardcover Price',fontsize = 20)
plt.savefig('./pics/Price Comparison between Hardcover and Paperback.png')


In [ ]:
std_valid = vaild_df[vaild_df['nyears'] > 2000]
std_valid = std_valid[std_valid['hand_new_price']<60]
std_valid = std_valid[std_valid['paper_new_price']<60]
std_valid['paper_new_price'] = std_valid['paper_new_price'].std()
std_valid['hand_new_price'] = std_valid['hand_new_price'].std()

plt.figure(figsize=(30,30))
plt.scatter(vaild_df['paper_new_price'],vaild_df['hand_new_price'],c = vaild_df['nyears'])

In [4]:
collindf = pd.read_csv("HarperCollins_nearest_3_years.csv")

UnicodeError: UTF-16 stream does not start with BOM